In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def show(*img):
    
    if len(img)<2:
        show_one(img[0])
        return
              
    fig, axes = plt.subplots(1, len(img))
    
    for img_i, i in zip(img, range(len(img))):
        axes[i].imshow(img_i, cmap='gray')
        axes[i].set_title('')

    fig.set_figwidth(20)    #  ширина и
    fig.set_figheight(10)    #  высота \"Figure\

#     plt.gray()
    plt.show()


def show_one(img, n = 10):
    fig, ax = plt.subplots()
    fig.set_figwidth(n)    #  ширина и
    fig.set_figheight(n)    #  высота \"Figure\
    plt.imshow(img, cmap='gray')
    plt.show()

In [ ]:
def preprocess_frame(frame):
    #     frame = frame[0:470, ]
    frame = cv2.GaussianBlur(frame, (11, 11), 0)

    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

#     v = cv2.equalizeHist(v)

    hsv = cv2.merge([h, s, v])
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)

    return bgr, v

In [ ]:
video_path = '90_2_3.mp4'

In [ ]:
import numpy as np
import cv2
import time
help_message = '''
USAGE: optical_flow.py [<video_source>]
Keys:
 1 - toggle HSV flow visualization
 2 - toggle glitch
'''
count = 0
def draw_flow(img, flow, step=16):
    h, w = img.shape[:2]
    y, x = np.mgrid[step/2:h:step, step/2:w:step].reshape(2,-1).astype(int)
    fx, fy = flow[y,x].T
    lines = np.vstack([x, y, x+fx, y+fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)
    # vis = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    cv2.polylines(img, lines, 0, (0, 255, 0))
    for (x1, y1), (_x2, _y2) in lines:
        cv2.circle(img, (x1, y1), 1, (0, 255, 0), -1)
    return img

def draw_hsv(flow):
    h, w = flow.shape[:2]
    fx, fy = flow[:,:,0], flow[:,:,1]
    ang = np.arctan2(fy, fx) + np.pi
    v = np.sqrt(fx*fx+fy*fy)
    hsv = np.zeros((h, w, 3), np.uint8)
    hsv[...,0] = ang*(180/np.pi/2)
    hsv[...,1] = 255
    hsv[...,2] = np.minimum(v*4, 255)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return bgr

def warp_flow(img, flow):
    h, w = flow.shape[:2]
    flow = -flow
    flow[:,:,0] += np.arange(w)
    flow[:,:,1] += np.arange(h)[:,np.newaxis]
    res = cv2.remap(img, flow, None, cv2.INTER_LINEAR)
    return res

In [ ]:

cam = cv2.VideoCapture(video_path)
ret, prev = cam.read()

prevgray = cv2.cvtColor(prev, cv2.COLOR_BGR2GRAY)
show_hsv = True
show_glitch = True
cur_glitch = prev.copy()

while True:
    ret, img = cam.read()
    vis = img.copy()
    frame, gray = preprocess_frame(img)
    flow = cv2.calcOpticalFlowFarneback(prevgray, gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    prevgray = gray
    if show_hsv:
        gray1 = cv2.cvtColor(draw_hsv(flow), cv2.COLOR_BGR2GRAY)
        show(gray1)
#         thresh = cv2.threshold(gray1, 25, 255, cv2.THRESH_BINARY)[1]
        thresh = cv2.adaptiveThreshold(gray1, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                          cv2.THRESH_BINARY, 199, 5)
        (cnts, _) = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        # loop over the contours
    for c in cnts:
        # if the contour is too small, ignore it
        (x, y, w, h) = cv2.boundingRect(c)
        if w > 10 and h > 10 and w < 900 and h < 680:
            cv2.drawContours(vis,c,-1,(0,255,0),3)
    show(draw_flow(gray, flow), thresh)
#     show(vis, cur_glitch)
    cur_glitch = warp_flow(cur_glitch, flow)

	